## Imports

In [ ]:
import sys
import os
import time

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "\\models")
    
from sklearn.pipeline import Pipeline
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.transformations.panel.compose import ColumnConcatenator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from KeyEventTimeSeries import KeyEventTimeSeries

In [ ]:
help(KeyEventTimeSeries)

## Load data

In [ ]:
ts = KeyEventTimeSeries()

ts.load("../data/facing_left_baseline.pkl")
ts.load("../data/facing_left_anticycle_baseline.pkl")
ts.load("../data/facing_left_index.pkl")
#ts.load("../data/facing_left_middle.pkl")
#ts.load("../data/facing_left_ring.pkl")
#ts.load("../data/facing_left_pinky.pkl")

## Dataset Info

In [ ]:
ts.data.sample(5)

In [ ]:
ts.data.info()

In [ ]:
ts.data.describe()

## Data preparation: Convert to sktime format
https://www.sktime.org/en/stable/

In [ ]:
df = ts.get_sktime_format(handedness="Left", r=range(-15, 5), include_curl=False, include_orientation=False)

In [ ]:
ts.stats()

In [ ]:
df.sample(5)

In [ ]:
df.loc[1, "Left Index Finger Distance"]

In [ ]:
ts.plot(col="Left Index Finger Distance")

## Forest Classifier
https://www.sktime.org/en/stable/get_started.html

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=5, n_jobs=-1)), # n=100 is default]
]
clf = Pipeline(steps)

# Set max depth
clf["classify"].base_estimator.max_depth = 12

clf.fit(X_train, y_train)
clf.score(X_test, y_test)